# Anexo D: Implementación

## Paquetes

### Paquetes de Julia

PyCall nos permite llamar a Python desde Julia

In [1]:
#using Pkg
#Pkg.add(PackageSpec(name="PyCall", rev="master"))
#Pkg.build("PyCall")
#Pkg.add("CSV")
#Pkg.add("Tables")

Ojo: Hay que pedirle dos ves a Julia que cargue el paquete PyCall (no se porque no lo carga a la primera).

In [3]:
using PyCall

In [4]:
using CSV, LinearAlgebra, Tables

### Paquetes de Python

In [5]:
py"""
import numpy as np
"""

## Métodos

A continuación se agregaran las funciones de Python y Julia, llamados desde Julia.

In [6]:
function JacJul(
        A::Array{Float64,2},
        b::Array{Float64,1},
        x0::Array{Float64,1},
        e::Float64,
        itermax::Int64)
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #e:= presicion
    #itermax:= iteración maxima
    
    #Iteración
    it::Int64 = 1
    
    #Matriz diagonal
    D::Array{Float64,2} = Diagonal(A)
    
    #Inversa de la matriz diagonal
    Dinv::Array{Float64,2} = inv(D)
    
    #Matriz triangular inferior
    L ::Array{Float64,2}= tril(A,-1)
    
    #Matriz triangular superior
    U::Array{Float64,2} = triu(A,1)
    
    #Iterando la solucion
    x::Array{Float64,1} = Dinv*(b- (L+U)*x0)
    
    #Calculando el residuo
    r::Array{Float64,1} = A*x-b
    
    #Normas de los residuos
    normresJAC::Array{Float64,1} = [norm(r)];
    
    #Repetir hasta el maximo numero de 
    #iteraciones o alcanzar la precision
    while ((normresJAC[it] > e) && (it <= itermax))      
        
        #Iterando la solucion
        x = Dinv*(b- (L+U)*x)
    
        #Calculando el residuo
        r = A*x-b
        
        #Normas de los residuos
        normresJAC = [normresJAC;norm(r)];
        
        #Aumentando la iteracion
        it = it + 1
        
    end
    
    return x, normresJAC
    
end;

In [7]:
py"""
def JacPy(A,b,x0,e,itermax):
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #e:= presicion
    #itermax:= iteración maxima
    
    #Iteración
    it = 1
    
    #Matriz diagonal
    D = np.diag(np.diag(A))
    
    #Inversa de la matriz diagonal
    Dinv = np.linalg.inv(D)
    
    #Matriz triangular superior
    U = np.triu(A, k=1)
    
    #Matriz triangular inferior
    L = np.tril(A, k=-1)
    
    #Iterando la solucion
    x = Dinv.dot(b - (L+U).dot(x0))
    
    #Calculando el residuo
    r = A.dot(x)- b
    
    #Normas de los residuos
    normresJAC = [np.linalg.norm(r,1)]
    
    #Repetir hasta el maximo numero de 
    #iteraciones o alcanzar la precision
    while ((normresJAC[it-1] > e) and (it <= itermax)):  
        
        #Iterando la solucion
        x = Dinv.dot(b - (L+U).dot(x))
    
        #Calculando la diferencia
        r = A.dot(x)- b
        
        #Normas de los residuos
        normresJAC = np.concatenate((normresJAC,
                                     np.linalg.norm(r,1)),
                                    axis = None)
        
        #Aumentando la iteracion
        it = it + 1
        
    return x, normresJAC
"""

In [8]:
function SRSJul(
        A::Array{Float64,2},
        b::Array{Float64,1},
        x0::Array{Float64,1},
        w::Float64,
        e::Float64,
        itermax::Int64)
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #w:= ponderador
    #e:= presicion
    #itermax:= iteración maxima
    
    #Iteración
    it::Int64 = 1

    #Matriz diagonal
    D::Array{Float64,2} = Diagonal(A)
    
    #Matriz triangular superior
    U::Array{Float64,2} = triu(A, 1)  
    
    #Matriz triangular inferior
    L::Array{Float64,2} = tril(A, -1)
    
    #alfa es una matriz auxiliar
    alfa::Array{Float64,2} = inv(w*L+D)
    
    #gama es un vector auxiliar
    gama::Array{Float64,1} = w*inv(D+w*L)*b
    
    #Iterando la solucion
    x::Array{Float64,1} = alfa*((1-w)*D*x0-w*U*x0) + gama
    
    #Calculando el residuo
    r::Array{Float64,1} = A*x-b
    
    #Normas de los residuos
    normresSCS::Array{Float64,1} = [norm(r)];
    
    #Repetir hasta el maximo numero de 
    #iteraciones o alcanzar la precision
    while ((normresSCS[it] > e) && (it <= itermax))    
        
        #Multiplicando
        x = alfa*((1-w)*D*x-w*U*x) + gama
    
        #Calculando la diferencia
        r = A*x-b
        
        #Normas de los residuos
        normresSCS = [normresSCS; norm(r)];

        #Aumentando la iteracion
        it = it + 1
    end
    
    return x, normresSCS
    
end;

In [9]:
py"""
def SRSPy(A,b,x0,w,e,itermax):
    
    #A:= Matriz
    #b:= vector
    #w:= ponderador
    #e:= presicion
    #itermax:= iteración máxima
    #x0:= inicialización
    
    #Iteración
    it = 1
    
    #Matriz diagonal
    D = np.diag(np.diag(A))
    
    #Matriz triangular superior
    U = np.triu(A, k=1)
    
    #Matriz triangular inferior
    L = np.tril(A, k=-1)
    
    #alfa es una matriz auxiliar
    alfa = np.linalg.inv(w*L+D)
    
    #gama es un vector auxiliar
    gama = w*np.linalg.inv(D+w*L).dot(b)
    
    #Iterando la solucion
    x = alfa.dot((1-w)*D.dot(x0)-w*U.dot(x0)) + gama
    
    #Calculando el residuo
    r = A.dot(x)-b
    
    #Normas de los residuos
    normresSCS = [np.linalg.norm(r,1)]
    
    #Repetir hasta el maximo numero de 
    #iteraciones o alcanzar la precision
    while ((normresSCS[it-1] > e) and (it <= itermax)): 
        
        #Multiplicando
        x = alfa.dot((1-w)*D.dot(x)-w*U.dot(x)) + gama
    
        #Calculando la diferencia
        r = A.dot(x)-b
        
        #Normas de los residuos
        normresSCS = np.concatenate((normresSCS,
                                     np.linalg.norm(r,1)),
                                    axis = None)
        
        #Aumentando la iteracion
        it = it + 1
        
                 
    return x, normresSCS
"""

In [10]:
function GSJul(
        A::Array{Float64,2},
        b::Array{Float64,1},
        x0::Array{Float64,1},
        e::Float64,
        itermax::Int64)
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #e:= presicion
    #itermax:= iteración maxima
    
    #Iteración
    it::Int64 = 1
    
    #Matriz triangular inferior
    L::Array{Float64,2} = tril(A)
    
    #Matriz triangular superior
    U::Array{Float64,2} = triu(A,1)
    
    #Iterando la solucion
    x::Array{Float64,1} = inv(L)*(b-U*x0);
    
    #Calculando el residuo
    r::Array{Float64,1} = b-A*x
    
    #Normas de los residuos
    normresGAUSS::Array{Float64,1} = [norm(r)];
    
    #Repetir hasta el maximo numero de 
    #iteraciones o alcanzar la precision
    while ((normresGAUSS[it] > e) && (it <= itermax))      
        
        #Iterando la solucion
        x = inv(L)*(b-U*x);
        
        #Calculando el residuo
        r = b - A*x
        
        #Normas de los residuos
        normresGAUSS = [normresGAUSS;norm(r)];
        
        #Aumentando la iteracion
        it = it + 1
        
    end

    return x, normresGAUSS
end;

In [11]:
py"""
def GSPy(A,b,x0,e,itermax):
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #e:= presicion
    #itermax:= iteración maxima
    
    #Iteración
    it = 1
    
    #Matriz triangular inferior
    U = np.triu(A, k=1)
    
    #Matriz triangular superior
    L = np.tril(A)
    
    #Iterando la solución
    x = np.linalg.inv(L).dot(b-U.dot(x0));
    
    #Calculando el residuo
    r = b-A.dot(x)
    
    #Respaldando la norma
    normresGS = [np.linalg.norm(r,1)]
    
    #Repetir hasta el maximo numero de 
    #iteraciones o alcanzar la precision
    while ((normresGS[it-1] > e) and (it <= itermax)):
        
        #Iterando la solucion
        x = np.linalg.inv(L).dot(b-U.dot(x));
        
        #Calculando el residuo
        r = b-A.dot(x)
        
        #Normas de los residuos
        normresGS = np.concatenate((normresGS,
                                np.linalg.norm(r,1)),
                               axis = None);
        
        #Aumentando la iteracion
        it = it + 1
    
    return x, normresGS
"""

In [12]:
function GCJul(
        A::Array{Float64,2},
        b::Array{Float64,1},
        x0::Array{Float64,1},
        e::Float64)
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #e:= presicion    
    
    #Iteración
    it::Int64 = 1
    
    #numeq := numero de ecuaciones
    numeq::Int64 = size(A,1)
    
    #Iterando la solucion
    x::Array{Float64,1} = x0;
    
    #Calculando el residuo
    r::Array{Float64,1} = b - A*x
    
    #Vectores auxiliares
    d::Array{Float64,1} = r;
    s::Array{Float64,1} = r;
    
    #Normas de los residuos
    normresGC::Array{Float64,1} = [norm(r)];
    
    #Constantes auxiliares
    a::Float64 = 0.0
    beta::Float64 = 0.0
    
    #Por cada ecuacion, hacer lo siguiente:
    for k in 1:numeq
       
        #Si la norma del vector es mayor a 
        #la presicion entonces
        if normresGC[it] > e
           
            #a:= constante auxiliar
            a = r'r/(d'*A*d)
            
            #Iterando la solucion
            x = x + a*d
            
            #s:= vector auxiliar
            s = r - a*A*d
            
            #beta:= constante auxiliar
            beta = s's/r'r
            
            #d:= vector auxiliar
            d = s + beta*d
            
            #actualizando los residuos
            r = s
            
            #Normas de los residuos
            normresGC = [normresGC; norm(r)];

        else
            return  x, normresGC
        end
        
    end
    
    return x, normresGC
end;

In [13]:
py"""
def GCPy(A,b,x0,e):
    
    #A:= Matriz de coeficientes
    #b:= vector de constantes
    #x0:= inicialización
    #e:= presicion    
    
    #Iteración
    it = 1
    
    #numeq := numero de ecuaciones
    numeq = A.shape[1]
    
    #Iterando la solucion
    x = x0;
    
    #Calculando el residuo
    r = b - A.dot(x)
    
    #Vectores auxiliares
    d = r;
    
    #Vector para guardar las convergencias
    normresGC = [np.linalg.norm(r,1)]
    
    #Por cada ecuacion, hacer lo siguiente:
    for k in range(numeq):
       
        #Si la norma del vector es mayor a 
        #la presicion entonces
        if normresGC[it-1] > e:
           
            #a:= constante auxiliar
            a = np.transpose(r).dot(r)/np.transpose(d).dot(A.dot(d))
            
            #Iterando la solucion
            x = x + a*d
            
            #s:= vector auxiliar
            s = r - a*A.dot(d)
            
            #beta:= constante auxiliar
            beta = np.transpose(s).dot(s)/np.transpose(r).dot(r)
            
            #d:= vector auxiliar
            d = s + beta*d
            
            #actualizando los residuos
            r = s
            
            #Normas de los residuos
            normresGC = np.concatenate((normresGC,
                                np.linalg.norm(r,1)),
                               axis = None);
            
            it = it + 1

        else:
            
            return  x, normresGC
        
    
    return x, normresGC
"""

In [14]:
function SistEQ(tam::Int64, x::Array{Float64,1})
    
    #tam:= tamanio del sistema de EQ
    #x:= solucion del sistema
    
    #Matriz de coeficientes (A)
    A::Array{Float64,2} = diagm(1 => -ones(tam-1),
                                0 => 3*ones(tam),
                               -1 => -ones(tam-1))
    
    for i in 1:tam
        if A[i,tam-i+1] == 0.0 
            A[i,tam-i+1] = 0.5
        end
    end
    
    #Vector de constantes (b)
    b::Array{Float64,1} = A*x
    
    return A,b
end;

## Evaluando los algoritmos e implementaciones

### Precisión fija (e)

In [15]:
e = 0.000001;

### Número máximo de iteraciones (itermax)

In [16]:
itermax = 100;

### Número de simulaciones

In [17]:
NSim = 300;

## Ejemplo

Se ejecutaron algunos ejemplos para que Julia constriya sus archivos temporales.

Primero se construiyen los parámetros de los algoritmos:

In [18]:
tam = 10;
A,b = SistEQ(tam,ones(tam));
x0 = zeros(tam);
w = 1.1;

Ejecutando los ejemplos:

In [19]:
@time sol,hist = JacJul(A,b,x0,e,itermax);

  0.225217 seconds (151.76 k allocations: 7.691 MiB)


In [20]:
@time sol,hist = py"JacPy"(A,b,x0,e,itermax);

  2.141659 seconds (2.15 M allocations: 109.166 MiB, 1.34% gc time)


In [21]:
@time sol,hist = SRSJul(A,b,x0,w,e,itermax);

  0.000085 seconds (414 allocations: 66.359 KiB)


In [22]:
@time sol,hist = py"SRSPy"(A,b,x0,w,e,itermax);

  0.032043 seconds (29.40 k allocations: 1.519 MiB)


In [23]:
@time sol,hist = GSJul(A,b,x0,e,itermax);

  0.000096 seconds (401 allocations: 45.031 KiB)


In [24]:
@time sol,hist = py"GSPy"(A,b,x0,e,itermax);

  0.001625 seconds (99 allocations: 4.672 KiB)


In [25]:
@time sol,hist = GCJul(A,b,x0,e);

  0.000066 seconds (254 allocations: 27.703 KiB)


In [26]:
@time sol,hist = py"GCPy"(A,b,x0,e);

  0.046043 seconds (29.36 k allocations: 1.518 MiB)


## Simulando

### Creando una tabla para respaldar los datos

Id:= Id de la simulación.

Nombre:= Nombre del algoritmo y simulación.

Ejecucion:= Julia ó Python.

N:= Tamaño de la entrada 20,40,50,80 ó 100.

Tiempo:= Tiempo de ejecución.

NumIte:= Número de iteraciones.

w:= Peso del método de sobre relajación.

In [27]:
Datos = Array{Any}(undef,5*18*NSim, 7);

Simulando

In [28]:
#Id del ejercicio.
id = 1

#n corre sobre el tamanio de la entrada
for n in [20,40,50,80,100]
    
    #Incializando la solución
    x0 = zeros(n)
    
    #Solución real
    x = ones(n)
    
    #Construyendo el sistema de ecuaciones
    A,b = SistEQ(n,x);
    
    #s cuenta el numero de simulaciones
    for s in 1:NSim
        
         #Ejecutando JacJul
        x0 = zeros(n)
        tiempo = @elapsed sol,hist = JacJul(A,b,x0,e,itermax);
        Datos[id,:] = [id, "JacJul", "Julia", n, tiempo, size(hist,1),0.0]
        id = id + 1
        
        #Ejecutando JacPy
        tiempo = @elapsed sol,hist = py"JacPy"(A,b,x0,e,itermax);
        Datos[id,:] = [id, "JacPy", "Python", n, tiempo, size(hist,1),0.0]
        id = id + 1
        
        
        for w in [1.0,1.2,1.4,1.6,1.8,2.0]
            
            #Ejecutando SRSJul
            tiempo = @elapsed sol,hist = SRSJul(A,b,x0,w,e,itermax);
            Datos[id,:] = [id, "SRSJul", "Julia", n, tiempo, size(hist,1),w]
            id = id + 1
        
            #Ejecutando SRSPy
            tiempo = @elapsed sol,hist = py"SRSPy"(A,b,x0,w,e,itermax);
            Datos[id,:] = [id, "SRSPy", "Python", n, tiempo, size(hist,1),w]
            id = id + 1
            
        end
        
        #Ejecutando GSJul
        tiempo = @elapsed sol,hist = GSJul(A,b,x0,e,itermax);
        Datos[id,:] = [id, "GSJul", "Julia", n, tiempo, size(hist,1),0.0]
        id = id + 1
        
        #Ejecutando GSPy
        tiempo = @elapsed sol,hist = py"GSPy"(A,b,x0,e,itermax);
        Datos[id,:] = [id, "GSPy", "Python", n, tiempo, size(hist,1),0.0]
        id = id + 1
        
        #Ejecutando GCJul
        tiempo = @elapsed sol,hist = GCJul(A,b,x0,e);
        Datos[id,:] = [id, "GCJul", "Julia", n, tiempo, size(hist,1),0.0]
        id = id + 1
        
        #Ejecutando GCJul
        tiempo = @elapsed sol,hist = py"GCPy"(A,b,x0,e);
        Datos[id,:] = [id, "GCPy", "Python", n, tiempo, size(hist,1),0.0]
        id = id + 1
        
        #Imprimiendo progreso
        println("Tamanio N = ",n," Núm de Sim s = ",s)
    end
    
end

Tamanio N = 20 Núm de Sim s = 1
Tamanio N = 20 Núm de Sim s = 2
Tamanio N = 20 Núm de Sim s = 3
Tamanio N = 20 Núm de Sim s = 4
Tamanio N = 20 Núm de Sim s = 5
Tamanio N = 20 Núm de Sim s = 6
Tamanio N = 20 Núm de Sim s = 7
Tamanio N = 20 Núm de Sim s = 8
Tamanio N = 20 Núm de Sim s = 9
Tamanio N = 20 Núm de Sim s = 10
Tamanio N = 20 Núm de Sim s = 11
Tamanio N = 20 Núm de Sim s = 12
Tamanio N = 20 Núm de Sim s = 13
Tamanio N = 20 Núm de Sim s = 14
Tamanio N = 20 Núm de Sim s = 15
Tamanio N = 20 Núm de Sim s = 16
Tamanio N = 20 Núm de Sim s = 17
Tamanio N = 20 Núm de Sim s = 18
Tamanio N = 20 Núm de Sim s = 19
Tamanio N = 20 Núm de Sim s = 20
Tamanio N = 20 Núm de Sim s = 21
Tamanio N = 20 Núm de Sim s = 22
Tamanio N = 20 Núm de Sim s = 23
Tamanio N = 20 Núm de Sim s = 24
Tamanio N = 20 Núm de Sim s = 25
Tamanio N = 20 Núm de Sim s = 26
Tamanio N = 20 Núm de Sim s = 27
Tamanio N = 20 Núm de Sim s = 28
Tamanio N = 20 Núm de Sim s = 29
Tamanio N = 20 Núm de Sim s = 30
Tamanio N = 20 Núm 

Tamanio N = 20 Núm de Sim s = 249
Tamanio N = 20 Núm de Sim s = 250
Tamanio N = 20 Núm de Sim s = 251
Tamanio N = 20 Núm de Sim s = 252
Tamanio N = 20 Núm de Sim s = 253
Tamanio N = 20 Núm de Sim s = 254
Tamanio N = 20 Núm de Sim s = 255
Tamanio N = 20 Núm de Sim s = 256
Tamanio N = 20 Núm de Sim s = 257
Tamanio N = 20 Núm de Sim s = 258
Tamanio N = 20 Núm de Sim s = 259
Tamanio N = 20 Núm de Sim s = 260
Tamanio N = 20 Núm de Sim s = 261
Tamanio N = 20 Núm de Sim s = 262
Tamanio N = 20 Núm de Sim s = 263
Tamanio N = 20 Núm de Sim s = 264
Tamanio N = 20 Núm de Sim s = 265
Tamanio N = 20 Núm de Sim s = 266
Tamanio N = 20 Núm de Sim s = 267
Tamanio N = 20 Núm de Sim s = 268
Tamanio N = 20 Núm de Sim s = 269
Tamanio N = 20 Núm de Sim s = 270
Tamanio N = 20 Núm de Sim s = 271
Tamanio N = 20 Núm de Sim s = 272
Tamanio N = 20 Núm de Sim s = 273
Tamanio N = 20 Núm de Sim s = 274
Tamanio N = 20 Núm de Sim s = 275
Tamanio N = 20 Núm de Sim s = 276
Tamanio N = 20 Núm de Sim s = 277
Tamanio N = 20

Tamanio N = 40 Núm de Sim s = 195
Tamanio N = 40 Núm de Sim s = 196
Tamanio N = 40 Núm de Sim s = 197
Tamanio N = 40 Núm de Sim s = 198
Tamanio N = 40 Núm de Sim s = 199
Tamanio N = 40 Núm de Sim s = 200
Tamanio N = 40 Núm de Sim s = 201
Tamanio N = 40 Núm de Sim s = 202
Tamanio N = 40 Núm de Sim s = 203
Tamanio N = 40 Núm de Sim s = 204
Tamanio N = 40 Núm de Sim s = 205
Tamanio N = 40 Núm de Sim s = 206
Tamanio N = 40 Núm de Sim s = 207
Tamanio N = 40 Núm de Sim s = 208
Tamanio N = 40 Núm de Sim s = 209
Tamanio N = 40 Núm de Sim s = 210
Tamanio N = 40 Núm de Sim s = 211
Tamanio N = 40 Núm de Sim s = 212
Tamanio N = 40 Núm de Sim s = 213
Tamanio N = 40 Núm de Sim s = 214
Tamanio N = 40 Núm de Sim s = 215
Tamanio N = 40 Núm de Sim s = 216
Tamanio N = 40 Núm de Sim s = 217
Tamanio N = 40 Núm de Sim s = 218
Tamanio N = 40 Núm de Sim s = 219
Tamanio N = 40 Núm de Sim s = 220
Tamanio N = 40 Núm de Sim s = 221
Tamanio N = 40 Núm de Sim s = 222
Tamanio N = 40 Núm de Sim s = 223
Tamanio N = 40

Tamanio N = 50 Núm de Sim s = 141
Tamanio N = 50 Núm de Sim s = 142
Tamanio N = 50 Núm de Sim s = 143
Tamanio N = 50 Núm de Sim s = 144
Tamanio N = 50 Núm de Sim s = 145
Tamanio N = 50 Núm de Sim s = 146
Tamanio N = 50 Núm de Sim s = 147
Tamanio N = 50 Núm de Sim s = 148
Tamanio N = 50 Núm de Sim s = 149
Tamanio N = 50 Núm de Sim s = 150
Tamanio N = 50 Núm de Sim s = 151
Tamanio N = 50 Núm de Sim s = 152
Tamanio N = 50 Núm de Sim s = 153
Tamanio N = 50 Núm de Sim s = 154
Tamanio N = 50 Núm de Sim s = 155
Tamanio N = 50 Núm de Sim s = 156
Tamanio N = 50 Núm de Sim s = 157
Tamanio N = 50 Núm de Sim s = 158
Tamanio N = 50 Núm de Sim s = 159
Tamanio N = 50 Núm de Sim s = 160
Tamanio N = 50 Núm de Sim s = 161
Tamanio N = 50 Núm de Sim s = 162
Tamanio N = 50 Núm de Sim s = 163
Tamanio N = 50 Núm de Sim s = 164
Tamanio N = 50 Núm de Sim s = 165
Tamanio N = 50 Núm de Sim s = 166
Tamanio N = 50 Núm de Sim s = 167
Tamanio N = 50 Núm de Sim s = 168
Tamanio N = 50 Núm de Sim s = 169
Tamanio N = 50

Tamanio N = 80 Núm de Sim s = 85
Tamanio N = 80 Núm de Sim s = 86
Tamanio N = 80 Núm de Sim s = 87
Tamanio N = 80 Núm de Sim s = 88
Tamanio N = 80 Núm de Sim s = 89
Tamanio N = 80 Núm de Sim s = 90
Tamanio N = 80 Núm de Sim s = 91
Tamanio N = 80 Núm de Sim s = 92
Tamanio N = 80 Núm de Sim s = 93
Tamanio N = 80 Núm de Sim s = 94
Tamanio N = 80 Núm de Sim s = 95
Tamanio N = 80 Núm de Sim s = 96
Tamanio N = 80 Núm de Sim s = 97
Tamanio N = 80 Núm de Sim s = 98
Tamanio N = 80 Núm de Sim s = 99
Tamanio N = 80 Núm de Sim s = 100
Tamanio N = 80 Núm de Sim s = 101
Tamanio N = 80 Núm de Sim s = 102
Tamanio N = 80 Núm de Sim s = 103
Tamanio N = 80 Núm de Sim s = 104
Tamanio N = 80 Núm de Sim s = 105
Tamanio N = 80 Núm de Sim s = 106
Tamanio N = 80 Núm de Sim s = 107
Tamanio N = 80 Núm de Sim s = 108
Tamanio N = 80 Núm de Sim s = 109
Tamanio N = 80 Núm de Sim s = 110
Tamanio N = 80 Núm de Sim s = 111
Tamanio N = 80 Núm de Sim s = 112
Tamanio N = 80 Núm de Sim s = 113
Tamanio N = 80 Núm de Sim s =

Tamanio N = 100 Núm de Sim s = 27
Tamanio N = 100 Núm de Sim s = 28
Tamanio N = 100 Núm de Sim s = 29
Tamanio N = 100 Núm de Sim s = 30
Tamanio N = 100 Núm de Sim s = 31
Tamanio N = 100 Núm de Sim s = 32
Tamanio N = 100 Núm de Sim s = 33
Tamanio N = 100 Núm de Sim s = 34
Tamanio N = 100 Núm de Sim s = 35
Tamanio N = 100 Núm de Sim s = 36
Tamanio N = 100 Núm de Sim s = 37
Tamanio N = 100 Núm de Sim s = 38
Tamanio N = 100 Núm de Sim s = 39
Tamanio N = 100 Núm de Sim s = 40
Tamanio N = 100 Núm de Sim s = 41
Tamanio N = 100 Núm de Sim s = 42
Tamanio N = 100 Núm de Sim s = 43
Tamanio N = 100 Núm de Sim s = 44
Tamanio N = 100 Núm de Sim s = 45
Tamanio N = 100 Núm de Sim s = 46
Tamanio N = 100 Núm de Sim s = 47
Tamanio N = 100 Núm de Sim s = 48
Tamanio N = 100 Núm de Sim s = 49
Tamanio N = 100 Núm de Sim s = 50
Tamanio N = 100 Núm de Sim s = 51
Tamanio N = 100 Núm de Sim s = 52
Tamanio N = 100 Núm de Sim s = 53
Tamanio N = 100 Núm de Sim s = 54
Tamanio N = 100 Núm de Sim s = 55
Tamanio N = 10

Tamanio N = 100 Núm de Sim s = 264
Tamanio N = 100 Núm de Sim s = 265
Tamanio N = 100 Núm de Sim s = 266
Tamanio N = 100 Núm de Sim s = 267
Tamanio N = 100 Núm de Sim s = 268
Tamanio N = 100 Núm de Sim s = 269
Tamanio N = 100 Núm de Sim s = 270
Tamanio N = 100 Núm de Sim s = 271
Tamanio N = 100 Núm de Sim s = 272
Tamanio N = 100 Núm de Sim s = 273
Tamanio N = 100 Núm de Sim s = 274
Tamanio N = 100 Núm de Sim s = 275
Tamanio N = 100 Núm de Sim s = 276
Tamanio N = 100 Núm de Sim s = 277
Tamanio N = 100 Núm de Sim s = 278
Tamanio N = 100 Núm de Sim s = 279
Tamanio N = 100 Núm de Sim s = 280
Tamanio N = 100 Núm de Sim s = 281
Tamanio N = 100 Núm de Sim s = 282
Tamanio N = 100 Núm de Sim s = 283
Tamanio N = 100 Núm de Sim s = 284
Tamanio N = 100 Núm de Sim s = 285
Tamanio N = 100 Núm de Sim s = 286
Tamanio N = 100 Núm de Sim s = 287
Tamanio N = 100 Núm de Sim s = 288
Tamanio N = 100 Núm de Sim s = 289
Tamanio N = 100 Núm de Sim s = 290
Tamanio N = 100 Núm de Sim s = 291
Tamanio N = 100 Núm 

### Exportando los datos

In [29]:
CSV.write("DatosProy2.csv",Tables.table(Datos),writeheader = false)

┌ Warning: `writeheader=false` is deprecated in favor of `header=false`
│   caller = ip:0x0
└ @ Core :-1


"DatosProy2.csv"